1. Import necessary libraries:

In [2]:
import numpy
import scipy
import pandas as pd
import math

2. Read dataset

In [3]:
df = pd.read_csv("Data/vehicles.csv")
df

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,7301591192,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,...,NaN,sedan,NaN,https://images.craigslist.org/00o0o_iiraFnHg8q...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:31-0600
426876,7301591187,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,...,NaN,sedan,red,https://images.craigslist.org/00x0x_15sbgnxCIS...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:29-0600
426877,7301591147,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,...,NaN,hatchback,white,https://images.craigslist.org/00L0L_farM7bxnxR...,Carvana is the safer way to buy a car During t...,NaN,wy,33.779214,-84.411811,2021-04-04T03:21:17-0600
426878,7301591140,https://wyoming.craigslist.org/ctd/d/atlanta-2...,wyoming,https://wyoming.craigslist.org,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,...,NaN,sedan,silver,https://images.craigslist.org/00z0z_bKnIVGLkDT...,Carvana is the safer way to buy a car During t...,NaN,wy,33.786500,-84.445400,2021-04-04T03:21:11-0600


3. Data Cleaning
Next we need to drop some uncorrelated columns. They are columns that we are pretty sure having no correlations to the car prices. As locations do not have major correlation to the prices, we choose to remove 'long', 'lat' and 'state' as well. I used Data Wrangler to investigate the dataset. 

In [ ]:
def clean_data(df):
    # Drop column: 'url'
    df = df.drop(columns=["url"])
    # Drop column: 'region_url'
    df = df.drop(columns=["region_url"])
    # Drop column: 'image_url'
    df = df.drop(columns=["image_url"])
    # Drop column: 'description'
    df = df.drop(columns=["description"])
    # Drop column: 'county'
    df = df.drop(columns=["county"])
    # Drop column: 'posting_date'
    df = df.drop(columns=["posting_date"])
    # Drop column: 'lat'
    df = df.drop(columns=["lat"])
    # Drop column: 'long'
    df = df.drop(columns=["long"])
    # Drop column: 'state'
    df = df.drop(columns=["state"])
    # Drop column: 'region'
    df = df.drop(columns=["region"])
    # Drop all other rows which are duplicated. Drop duplicate rows in columns: 'price', 'year' and 13 other columns
    df = df.drop_duplicates(
        subset=[
            "price",
            "year",
            "manufacturer",
            "model",
            "condition",
            "cylinders",
            "fuel",
            "odometer",
            "title_status",
            "transmission",
            "VIN",
            "drive",
            "size",
            "type",
            "paint_color",
        ]
    )
    # Some cars with the same VIN and other characteristics have different models. 
    # Lets drop this cars too. Drop duplicate rows in columns: 'price', 'year' and 12 other columns
    df = df.drop_duplicates(
        subset=[
            "price",
            "year",
            "manufacturer",
            "condition",
            "cylinders",
            "fuel",
            "odometer",
            "title_status",
            "transmission",
            "VIN",
            "drive",
            "size",
            "type",
            "paint_color",
        ]
    )
    # Drop all duplicated VIN raws which are not null
    filtered_df = df[df["VIN"].notnull()]
    duplicated_df = filtered_df[filtered_df.duplicated(subset="VIN", keep=False)]
    df = df.drop(duplicated_df.index)
    # Now all VIN columns are unique and we don't need this column any more, we are going to use only id column.
    # Drop column: 'VIN'
    df = df.drop(columns=["VIN"])
    # Drop duplicate rows in columns: 'price', 'year' and 12 other columns
    df = df.drop_duplicates(
        subset=[
            "price",
            "year",
            "manufacturer",
            "model",
            "condition",
            "cylinders",
            "fuel",
            "odometer",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
        ]
    )
    return df


df_clean = clean_data(df.copy())
df_clean

4. Filling up null values and removing outliers 
Type and manufacturer columns have missing values. Lets rewrite missing type and manufacturer values according to the model.

In [4]:
def modef(x):  # get mode of groupby row
    m = pd.Series.mode(x)
    if len(m) == 1:
        return m
    if len(m) == 0:
        return "unknown"
    else:
        return m[0]


def fill_type(
    x, model_dict, column
):  # fill type column with mode of model columns, add info about NaN models
    if pd.isnull(x[column]):
        try:
            out = model_dict[
                x["model"]
            ]  # dictionary! model_dict with type or manufacturer values
        except:
            out = "unknown"
    else:
        out = x[column]
    return out


model_types = df_clean.groupby(["model"])["type"].agg(modef)
# Filling missing type values 
df_clean["type"] = df_clean.apply(fill_type, args=(model_types, "type"), axis=1)


In [5]:
# filling missing manufacturer values
model_manufacturer = df_clean.groupby(['model'])['manufacturer'].agg(modef)
df_clean['manufacturer'] = df_clean.apply(fill_type, args=(model_manufacturer, 'manufacturer'), axis=1)

Lets remove car outliers by price and odometer.

In [ ]:
# removing price and odometer outliers 
def clean_data_0(df_clean):
    # Filter rows based on column: 'price'
    df_clean = df_clean[(df_clean['price'] > 100) & (df_clean['price'] < 100000)]
    df_clean = df_clean[df_clean['odometer'] < 1500000]
    return df_clean

df_clean_0 = clean_data_0(df_clean.copy())
df_clean_0



There are "condition" missing values. We can find condition values from odometer and year. 

In [7]:
df_clean_1 = df_clean_0.copy()

#Finding mean odometer readings for each condition type
excellent_odo_mean = df_clean_1[df_clean_1['condition'] == 'excellent']['odometer'].mean()
good_odo_mean = df_clean_1[df_clean_1['condition'] == 'good']['odometer'].mean()
like_new_odo_mean = df_clean_1[df_clean_1['condition'] == 'like new']['odometer'].mean()
salvage_odo_mean = df_clean_1[df_clean_1['condition'] == 'salvage']['odometer'].mean()
fair_odo_mean = df_clean_1[df_clean_1['condition'] == 'fair']['odometer'].mean()

print('Like new average odometer:', round( like_new_odo_mean,2))
print('Excellent average odometer:', round( excellent_odo_mean,2))
print('Good average odometer:', round( good_odo_mean,2))
print('Fair average odometer:', round( fair_odo_mean,2))
print('Salvage average odometer:', round( salvage_odo_mean,2))

df_clean_1.loc[df_clean_1.year>=2019, 'condition'] = df_clean_1.loc[df_clean_1.year>=2019, 'condition'].fillna('new')


df_clean_1.loc[df_clean_1['odometer'] <= like_new_odo_mean, 'condition'] = df_clean_1.loc[df_clean_1['odometer'] <= like_new_odo_mean, 'condition'].fillna('like new')

df_clean_1.loc[df_clean_1['odometer'] >= fair_odo_mean, 'condition'] = df_clean_1.loc[df_clean_1['odometer'] >= fair_odo_mean, 'condition'].fillna('fair')

df_clean_1.loc[((df_clean_1['odometer'] > good_odo_mean) & 
       (df_clean_1['odometer'] <= excellent_odo_mean)), 'condition'] = df_clean_1.loc[((df_clean_1['odometer'] > good_odo_mean) & 
       (df_clean_1['odometer'] <= excellent_odo_mean)), 'condition'].fillna('excellent')

df_clean_1.loc[((df_clean_1['odometer'] > like_new_odo_mean) & 
       (df_clean_1['odometer'] <= good_odo_mean)), 'condition'] = df_clean_1.loc[((df_clean_1['odometer'] > like_new_odo_mean) & 
       (df_clean_1['odometer'] <= good_odo_mean)), 'condition'].fillna('good')

df_clean_1.loc[((df_clean_1['odometer'] > good_odo_mean) & 
       (df_clean_1['odometer'] <= fair_odo_mean)), 'condition'] = df_clean_1.loc[((df_clean_1['odometer'] > good_odo_mean) & 
       (df_clean_1['odometer'] <= fair_odo_mean)), 'condition'].fillna('salvage')
df_clean_1

Like new average odometer: 82395.05
Excellent average odometer: 108915.9
Good average odometer: 123733.17
Fair average odometer: 172738.23
Salvage average odometer: 162958.63


,id,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
27,7316814884,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,NaN,NaN,pickup,white
28,7316814758,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,NaN,NaN,pickup,blue
29,7316814989,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,NaN,NaN,pickup,red
30,7316743432,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,NaN,NaN,pickup,red
31,7316356412,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426831,7302384818,9800,1985.0,nissan,300zx coupe with t-tops,like new,6 cylinders,gas,115000.0,clean,automatic,rwd,sub-compact,hatchback,red
426833,7302338378,6800,1997.0,jaguar,xk8 convertible,good,8 cylinders,gas,69550.0,clean,automatic,rwd,compact,convertible,white
426838,7302198494,21900,1920.0,unknown,Paige Glenbrook Touring,good,6 cylinders,gas,11065.0,clean,manual,rwd,full-size,other,black
426839,7302191669,54999,2017.0,unknown,2017,good,NaN,gas,119000.0,clean,automatic,NaN,NaN,sedan,NaN


The condition values are being filled by the values from above calculated mean value ranges.

I am dropping the nan values of remaining columns.

In [ ]:
def clean_data_1(df_clean_1):
    # Drop rows with missing data in column: 'drive'
    df_clean_1 = df_clean_1.dropna(subset=['drive'])
    # Drop rows with missing data in column: 'transmission'
    df_clean_1 = df_clean_1.dropna(subset=['transmission'])
    # Drop rows with missing data in column: 'title_status'
    df_clean_1 = df_clean_1.dropna(subset=['title_status'])
    # Drop rows with missing data in column: 'year'
    df_clean_1 = df_clean_1.dropna(subset=['year'])
    # Drop rows with missing data in column: 'model'
    df_clean_1 = df_clean_1.dropna(subset=['model'])
    # Drop rows with missing data in column: 'fuel'
    df_clean_1 = df_clean_1.dropna(subset=['fuel'])
    # Drop rows with missing data in column: 'cylinders'
    df_clean_1 = df_clean_1.dropna(subset=['cylinders'])
    # Replace missing values with "unknown" in column: 'size'
    df_clean_1 = df_clean_1.fillna({'size': "unknown"})
    # Replace missing values with "unknown" in column: 'paint_color'
    df_clean_1 = df_clean_1.fillna({'paint_color': "unknown"})
    return df_clean_1

df_clean_2 = clean_data_1(df_clean_1.copy())
df_clean_2

However, there are some relatively new cars were sold nearly for free, which is against common sense. Thus we need to do some intervention to it. 
Cars that are too old (let's say earier than 1950) will increase uncertainty to our data prediction, because of the insufficient amount and probably unstable prices (some of them can be regarded as antiques)

In [9]:
df_clean_2 = df_clean_2[(df_clean_2['price']+df_clean_2['odometer'])>5000]
df_clean_2 = df_clean_2[df_clean_2['year']>1950]

Some brands have too few samples.
We remove the manufacturers which hold less than 100 records. And models with less then 30 records.

In [10]:
df_clean_2['manufacturer'].value_counts()

manufacturer
ford               19815
chevrolet          16142
toyota             11319
honda               7825
jeep                6047
nissan              5943
unknown             4326
gmc                 4219
ram                 3874
dodge               3832
bmw                 3656
subaru              3249
mercedes-benz       2989
hyundai             2985
volkswagen          2799
kia                 2214
lexus               2137
chrysler            2088
cadillac            1655
mazda               1629
audi                1625
buick               1544
acura               1213
infiniti            1053
pontiac              999
volvo                946
lincoln              937
mini                 714
mitsubishi           696
mercury              549
rover                530
saturn               419
porsche              410
jaguar               350
fiat                 208
alfa-romeo            52
tesla                 42
datsun                30
harley-davidson       23
ferrari     

In [11]:
df_clean_2 = df_clean_2.groupby('manufacturer').filter(lambda x: len(x) > 100)
df_clean_2 = df_clean_2.groupby('model').filter(lambda x: len(x) > 30)
#df_clean_2['manufacturer'].value_counts()
df_clean_2['model'].value_counts()

model
f-150                 2177
silverado 1500        1628
accord                1199
camry                 1184
1500                  1136
                      ... 
milan                   31
pathfinder se           31
e-350                   31
outback 2.5i            31
expedition limited      31
Name: count, Length: 577, dtype: int64

In [12]:
df_clean_2

,id,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
31,7316356412,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black
34,7316285779,35000,2019.0,toyota,tacoma,like new,6 cylinders,gas,43000.0,clean,automatic,4wd,unknown,truck,grey
37,7316130053,4500,1992.0,jeep,cherokee,excellent,6 cylinders,gas,192000.0,clean,automatic,4wd,unknown,SUV,unknown
38,7315816316,32990,2017.0,jeep,wrangler unlimited sport,good,6 cylinders,gas,30041.0,clean,other,4wd,unknown,other,silver
55,7314560853,19900,2004.0,ford,f250 super duty,good,8 cylinders,diesel,88000.0,clean,automatic,4wd,full-size,pickup,blue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426737,7304360046,2500,2005.0,honda,civic,good,4 cylinders,gas,185000.0,clean,manual,fwd,compact,sedan,grey
426744,7303806527,800,2000.0,cadillac,deville,excellent,8 cylinders,gas,100000.0,clean,automatic,rwd,full-size,coupe,white
426746,7303723154,17950,2004.0,ford,f350 super duty,excellent,8 cylinders,diesel,186000.0,clean,automatic,4wd,unknown,other,unknown
426760,7303500610,6250,2006.0,chevrolet,silverado 2500hd,good,8 cylinders,gas,200000.0,clean,automatic,4wd,full-size,truck,red
